# OpenAI - fast batch processing

The guide is a companion to the paper *"Generative LLMs and Textual Analysis in Accounting:(Chat)GPT as Research Assistant?"* ([SSRN](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4429658))

**Author:** [Ties de Kok](https://www.tiesdekok.com)    

----
# Imports
----


All the dependencies required for this notebook are provided in the `environment.yml` file.

To install: `conda env create -f environment.yml` --> this creates the `gllm` environment.

I recommend using Python 3.9 or higher to avoid dependency conflicts.

**Python built-in libraries**

In [1]:
import os, sys, re, copy, random, json, time, datetime
from pathlib import Path
import getpass

**Libraries for interacting with the OpenAI API**

In [2]:
import requests
import openai
import tiktoken

**General helper libraries**

In [3]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

### Import custom async logic

This is a custom implementation I created based on the async example code by OpenAI. This code maximizes the token throughput using async requests to the OpenAI API. 

**Warning!** This function is experimental and might break, use at your own risk and discretion

In [4]:
import fast_openai_async
make_batch_predictions = fast_openai_async.make_batch_predictions

### Settings

In [5]:
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

### Utility functions

In [6]:
## This function makes it easier to print rendered markdown through a code cell.

from IPython.display import Markdown

def mprint(text, *args, **kwargs):
    if 'end' in kwargs.keys():
        text += kwargs['end']
        
    display(Markdown(text))

-----
# Toy example
----

I will solve the exact same problem as the "zero_shot.ipynb" file.

## Create prompt dataset

#### Load dataset

In [9]:
with open(Path.cwd() / "data" / "statements.json", "r", encoding = "utf-8") as f:
    statement_list = json.load(f)

statement_df = pd.DataFrame(statement_list)

#### Define prompt template

In [10]:
prompt_template = """
Task: classify whether the statement below contains a forward looking statements (fls).
Rules:
- Answer using JSON in the following format: {{"contains_fls" : 0 or 1}}
Statement:
> {statement}
JSON =
""".strip()

#### Create dataset to make predictions for

In [15]:
prompt_list = []
for i, row in statement_df.iterrows():
    prompt = prompt_template.format(**{
                "statement" : row["statement"]
            })
    
    prompt_list.append({
        "id" : row["i"], ## This is important to merge the results back.
        "prompt" : prompt
    })
len(prompt_list)

60

## Generate predictions

**Important note:** the rate limits are hard-coded values in the `fast_openai_async.py` file and you might need to update them to reflect the rate limits shown on your OpenAI account page:

https://platform.openai.com/account/rate-limits

In [25]:
RUN = True
if RUN:
    print("Start time is:", datetime.datetime.now())
    _ = await make_batch_predictions(
        prompt_list,
        "gpt-3.5-turbo", # e.g., "gpt-4", "gpt-3.5-turbo-0613", etc
        Path.cwd() / "data",
        Path.cwd() / "data",
        max_tokens = 600,
        #system_message = system_message, ## Not specifying it sets it to default
        return_json = False,
        print_interval = 10,
        label = f"demo_run_v1",
        prompt_template = prompt_template,
        store_prompt_loc = Path.cwd() / "data"
    )
    print("End time is:", datetime.datetime.now())

INFO:root:Starting request #0
INFO:root:Starting request #10
INFO:root:Starting request #20
INFO:root:Starting request #30
INFO:root:Starting request #40
INFO:root:Starting request #50


Start time is: 2023-10-04 16:49:42.001683
Running with the following rate limits: 3,395 RPM and 79,200 TPM
Submitting 60 jobs for gpt-3.5-turbo!
Results are saved to the following file: C:/Users/kokti/Dropbox/Work/Research/chatgpt_paper/github/chatgpt_paper/code_examples/openai/data/results_20231004164942_demo_run_v1.jsonl


INFO:root:Parallel processing complete. Results saved to C:/Users/kokti/Dropbox/Work/Research/chatgpt_paper/github/chatgpt_paper/code_examples/openai/data/results_20231004164942_demo_run_v1.jsonl


Done, the results are available in the following file:
C:/Users/kokti/Dropbox/Work/Research/chatgpt_paper/github/chatgpt_paper/code_examples/openai/data/results_20231004164942_demo_run_v1.jsonl
End time is: 2023-10-04 16:49:43.473135


## Process results

#### Load in results

In [19]:
res_file = "C:/Users/kokti/Dropbox/Work/Research/chatgpt_paper/github/chatgpt_paper/code_examples/openai/data/results_20231004164533_demo_run_v1.jsonl"

success_res, error_res = fast_openai_async.proc_results(res_file, "chat")

In [22]:
success_res[:3]

[{'contains_fls': 1, 'id': 2},
 {'contains_fls': 1, 'id': 12},
 {'contains_fls': 0, 'id': 13}]

#### Inspect tokens used

**Important note:** these calculations are done using hard-coded values in the `fast_openai_async.py` file and are only up-to-date as per September 2023. 
You might need to update them!

In [24]:
fast_openai_async.calc_tokens_used(res_file)

Cost estimates:

- gpt-4 - $0.17
- gpt-35-4k - $0.01
- gpt-35-16k - $0.02
- gpt-35-ft - $0.06

Total number of tokens used: 5,141


,prompt_tokens,completion_tokens,total_tokens,qanda_id
count,60.000000,60.0,60.000000,60.000000
mean,76.683333,9.0,85.683333,30.500000
std,3.855402,0.0,3.855402,17.464249
min,68.000000,9.0,77.000000,1.000000
25%,74.000000,9.0,83.000000,15.750000
50%,77.000000,9.0,86.000000,30.500000
75%,79.000000,9.0,88.000000,45.250000
max,88.000000,9.0,97.000000,60.000000


#### Add results back to Pandas dataframe

In [32]:
res_df = pd.DataFrame(success_res)

In [33]:
res_df = res_df.rename(columns = {
    "id" : "i",
    "contains_fls" : "fls_prediction"
})

In [35]:
combo_df = pd.merge(statement_df, res_df, on = "i", how = "left")

In [36]:
combo_df.head(10)

,i,statement,contains_fls,fls_prediction
0,1,"In the last quarter, we managed to increase ou...",0,0
1,2,We anticipate that our investments in R&D will...,1,1
2,3,Our recent acquisition of XYZ Company has alre...,0,0
3,4,We expect to see continued growth in the Asian...,1,1
4,5,"In the past year, we have successfully reduced...",0,0
5,6,"Looking ahead, we're projecting a 5% increase ...",1,1
6,7,The launch of our latest software solution las...,0,0
7,8,We believe the introduction of our new AI-powe...,1,1
8,9,Our strong performance this year can be attrib...,0,0
9,10,"By expanding our sales team, we are confident ...",1,1


#### Evaluate performance

In [37]:
from sklearn.metrics import classification_report

In [38]:
print(classification_report(
    combo_df["contains_fls"], 
    combo_df["fls_prediction"]
))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        30
           1       1.00      0.97      0.98        30

    accuracy                           0.98        60
   macro avg       0.98      0.98      0.98        60
weighted avg       0.98      0.98      0.98        60

